In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Part 1
<hr>

In [2]:
# Get data and parse using beautiful soap
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'xml')

### Preprocessing

In [3]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

### Integrating Postal Codes

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


### Creating Pandas Dataframe

In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M8W,Etobicoke,"Alderwood, Long Branch"
1,M4N,Central Toronto,Lawrence Park
2,M6H,West Toronto,Dovercourt Village
3,M2L,North York,York Mills
4,M2H,North York,Hillcrest Village
5,M1V,Scarborough,"Agincourt North, Milliken"
6,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
7,M5E,Downtown Toronto,Berczy Park
8,M4X,Downtown Toronto,"Cabbagetown, St. James Town"
9,M8V,Etobicoke,New Toronto


# Part 2
<hr/>

In [6]:
#add Geo-spatial data
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df_toronto.set_index("Postcode")
df_toronto=pd.merge(df_toronto, dfll)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
3,M2L,North York,York Mills,43.757490,-79.374714
4,M2H,North York,Hillcrest Village,43.803762,-79.363452
5,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577
6,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
8,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
9,M8V,Etobicoke,New Toronto,43.605647,-79.501321


# Part 3 
<hr>

In [18]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [19]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [32]:
CLIENT_ID = 'TP5G4DYKRUJUJP5SHJIEOTLBNQJNBJA4CKRO4GTDPR3TS1PQ' # your Foursquare ID
CLIENT_SECRET = 'CW5LCVOUJQ1A4WLKIVISPA3VPU3GKMDCNMHX5PKAAEB1CKLQ' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version

In [33]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
3,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
5,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
6,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029


In [34]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [35]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

In [36]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Clairlea, Golden Mile, Oakridge" are 43.711111700000004, -79.2845772.


In [37]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3fed984eefd00025504078'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'contact': {},
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.25683

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Clairlea, Golden Mile, Oakridge
Rouge, Malvern
Scarborough Village
Maryvale, Wexford
Highland Creek, Rouge Hill, Port Union
Dorset Park, Scarborough Town Centre, Wexford Heights
Ionview, Kennedy Park
Tam O'Shanter
Agincourt
Birch Cliff
Morningside, West Hill
Upper Rouge
Cliffcrest, Cliffside
Agincourt North, Milliken
Woburn


In [44]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Dairy Queen,43.710378,-79.290701,Fast Food Restaurant
1,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Ave & St. Clair Ave E,43.712057,-79.281005,Intersection
2,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus #68 Warden,43.711778,-79.279714,Bus Line
3,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Subway Station,43.711229,-79.279602,Metro Station
4,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus 102 Markham Road,43.711381,-79.279588,Bus Line
5,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Station Bus Loop,43.711241,-79.279576,Bus Station
6,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Bakery On The Go,43.711271,-79.279506,Bakery
7,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Cafe on the go,43.711151,-79.279469,Bakery
8,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Woods Park,43.710527,-79.278966,Park
9,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Clairlea Futbol Centre,43.715234,-79.286506,Soccer Field


In [46]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"Ionview, Kennedy Park",5,5,5,5,5,5
"Maryvale, Wexford",7,7,7,7,7,7


In [49]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Golf Course,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Golf Course,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.00,0.000000,0.25,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.0,0.5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000
2,Birch Cliff,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.000000,0.00,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.0,0.0,0.2,0.0,0.0,0.00,0.2,0.1,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.000000,0.1,0.0,0.0,0.0,0.000000,0.00,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.000000
4,"Cliffcrest, Cliffside",0.5,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.166667,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.333333,0.0,0.0,0.0,0.0,0.166667,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.166667
6,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.5,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000


### Group Venues per Neighbor

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Agincourt North, Milliken",Park,Playground,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Park,Fast Food Restaurant,Metro Station,Bus Station,Intersection,Soccer Field,Breakfast Spot,Bar
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Intersection,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"Highland Creek, Rouge Hill, Port Union",Golf Course,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,"Ionview, Kennedy Park",Coffee Shop,Discount Store,Bus Station,Department Store,Convenience Store,Vietnamese Restaurant,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
8,"Maryvale, Wexford",Auto Garage,Smoke Shop,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint
9,"Morningside, West Hill",Intersection,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Fried Chicken Joint


### Clustering into 5 clusters

In [63]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 3, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [64]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1,Bus Line,Bakery,Park,Fast Food Restaurant,Metro Station,Bus Station,Intersection,Soccer Field,Breakfast Spot,Bar
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Golf Course,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Jewelry Store,Playground,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
3,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,1,Auto Garage,Smoke Shop,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Golf Course,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,4,Coffee Shop,Discount Store,Bus Station,Department Store,Convenience Store,Vietnamese Restaurant,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,1,Pizza Place,Italian Restaurant,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Noodle House,Bank,Shopping Mall
8,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Visualizing the Clusters

In [65]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters